In [ ]:
%pip install pandas
%pip install numpy
%pip install scipy
%pip install plotnine

In [ ]:
import pandas as pd
import numpy as np
import scipy.integrate as integrate
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def utilcalc(b1, b2, theta_upper, theta_lower, fc=1, fd=1, pi=0.5):

  b_bar = (b1+b2)/2
  # print("b_bar =", b_bar)

  delt = (theta_upper-theta_lower)/2
  # print("delt =", delt)

  # cut points for integrals

  # thea1:
  # Min{b_bar, b1+theta_upper}
  min_bb_b1thu = np.min([b_bar, (b1+theta_upper)])
  # Min{b_bar, b1+theta_lower}
  min_bb_b1thl = np.min([b_bar, (b1+theta_lower)])
  # Min{b_bar-delta, b1+theta_lower}
  min_bbmd_b1thl = np.min([(b_bar-delt), (b1+theta_lower)])
  # Min{b_bar+delta, b1+theta_upper}
  min_bbpd_b1thu = np.min([(b_bar+delt), (b1+theta_upper)])

  # Max{0, b1-theta_upper}
  max_0_b1thu = np.max([0, (b1-theta_upper)])
  # Max{0, b1-theta_lower}
  max_0_b1thl = np.max([0, (b1-theta_lower)])

  # thea2:
  # Max{b_bar, b2-theta_upper}
  max_bb_b2thu = np.max([b_bar, (b2-theta_upper)])
  # Max{b_bar, b2-theta_lower}
  max_bb_b2thl = np.max([b_bar, (b2-theta_lower)])
  # Max{b_bar+delta, b2-theta_lower}
  max_bbpd_b2thl = np.max([(b_bar+delt), (b2-theta_lower)])
  # Max{b_bar-delta, b2-theta_upper}
  max_bbmd_b2thu = np.max([(b_bar-delt), (b2-theta_upper)])

  # Min{1, b2+theta_upper}
  min_1_b2thu = np.min([1, (b2+theta_upper)])
  # Min{1, b2+theta_lower}
  min_1_b2thl = np.min([1, (b2+theta_lower)])

  u1cc = (pi*np.max([(integrate.quad(lambda x: fc, max_0_b1thu, min_bb_b1thu))[0],0])) + ((1-pi)*np.max([(integrate.quad(lambda x: fd, max_0_b1thl, min_bb_b1thl))[0],0]))
  # print("U1CC =", u1cc)

  u2cc = (pi*np.max([(integrate.quad(lambda x: fc, max_bb_b2thu, min_1_b2thu))[0],0])) + ((1-pi)*np.max([(integrate.quad(lambda x: fd, max_bb_b2thl, min_1_b2thl))[0],0]))
  # print("U2CC =", u2cc)

  u1dd = (pi*np.max([(integrate.quad(lambda x: fc, max_0_b1thl, min_bb_b1thl))[0],0])) + ((1-pi)*np.max([(integrate.quad(lambda x: fd, max_0_b1thu, min_bb_b1thu))[0],0]))
  # print("U1DD =", u1dd)

  u2dd = (pi*np.max([(integrate.quad(lambda x: fc, max_bb_b2thl, min_1_b2thl))[0],0])) + ((1-pi)*np.max([(integrate.quad(lambda x: fd, max_bb_b2thu, min_1_b2thu))[0],0]))
  # print("U2DD =", u2dd)

  u1cd = (pi*np.max([(integrate.quad(lambda x: fc, max_0_b1thu, min_bbpd_b1thu))[0],0])) + ((1-pi)*np.max([(integrate.quad(lambda x: fd, max_0_b1thl, min_bbmd_b1thl))[0],0]))
  # print("U1CD =", u1cd)

  u2cd = (pi*np.max([(integrate.quad(lambda x: fc, max_bbpd_b2thl, min_1_b2thl))[0],0])) + ((1-pi)*np.max([(integrate.quad(lambda x: fd, max_bbmd_b2thu, min_1_b2thu))[0],0]))
  # print("U2CD =", u2cd)

  u1dc = (pi*np.max([(integrate.quad(lambda x: fc, max_0_b1thl, min_bbmd_b1thl))[0],0])) + ((1-pi)*np.max([(integrate.quad(lambda x: fd, max_0_b1thu, min_bbpd_b1thu))[0],0]))
  # print("U1DC =", u1dc)

  u2dc = (pi*np.max([(integrate.quad(lambda x: fc, max_bbmd_b2thu, min_1_b2thu))[0],0])) + ((1-pi)*np.max([(integrate.quad(lambda x: fd, max_bbpd_b2thl, min_1_b2thl))[0],0]))
  # print("U2DC =", u2dc)

  return {"u1cc":round(u1cc, 4), "u2cc":round(u2cc, 4), "u1dd":round(u1dd, 4), "u2dd":round(u2dd, 4), "u1cd":round(u1cd, 4), "u2cd":round(u2cd, 4), "u1dc":round(u1dc, 4), "u2dc":round(u2dc, 4)}

In [ ]:
def mixing(u1cc, u2cc, u1dd, u2dd, u1cd, u2cd, u1dc, u2dc):
  p1 = (u2dd-u2dc)/(u2cc+u2dd-u2cd-u2dc)
  p2 = (u1dd-u1cd)/(u1cc+u1dd-u1dc-u1cd)

  return {"p1": round(p1, 4), "p2": round(p2, 4)}

In [ ]:
def equilib_checker(u1cc, u2cc, u1dd, u2dd, u1cd, u2cd, u1dc, u2dc, p1, p2):
  cc = False
  dd = False
  cd = False
  dc = False
  mixed = False

  if (u1cc >= u1dc) & (u2cc >= u2cd):
    cc = True
  if (u1dd >= u1cd) & (u2dd >= u2dc):
    dd = True
  if (u1cd >= u1dd) & (u2cd >= u2cc):
    cd = True
  if (u1dc >= u1cc) & (u2dc >= u2dd):
    dc = True

  if (p1 > 0) & (p1 < 1) & (p2 > 0) & (p2 < 1):
    mixed = True

  return {"CC":cc, "DD":dd, "CD":cd, "DC":dc, "Mixed":mixed}

In [ ]:
fc = 1
fd = 1
pi = 0

pi_inc = 0.025
theta_inc = 0.025
b_inc = 0.025

b_values = np.arange(0, 1 + b_inc, b_inc)
print(b_values)
theta_values = np.arange(0, 1 + theta_inc, theta_inc)
print(theta_values)
pi_values = np.arange(0, 1 + pi_inc, pi_inc)
print(pi_values)

df = pd.DataFrame(columns=["b1", "b2", "theta_upper", "theta_lower", "fc", "fd", "pi",
                           "u1cc", "u2cc", "u1dd", "u2dd", "u1cd", "u2cd", "u1dc", "u2dc",
                           "CC", "DD", "CD", "DC", "P1", "P2", "Mixed"])


for pi in pi_values:
    for b1 in b_values:
        for b2 in b_values:
            if b2 > b1:
                for theta_lower in theta_values:
                    for theta_upper in theta_values:
                        if theta_upper > theta_lower:
                            a = utilcalc(b1=b1, b2=b2, theta_upper=theta_upper, theta_lower=theta_lower, fc=1, fd=1, pi=pi)
                            b = mixing(a["u1cc"], a["u2cc"], a["u1dd"], a["u2dd"], a["u1cd"], a["u2cd"], a["u1dc"], a["u2dc"])
                            c = equilib_checker(a["u1cc"], a["u2cc"], a["u1dd"], a["u2dd"], a["u1cd"], a["u2cd"], a["u1dc"], a["u2dc"], b["p1"], b["p2"])
                            tempdf = pd.DataFrame.from_dict({"b1": [b1], "b2": [b2], "theta_upper": [theta_upper], "theta_lower": [theta_lower], "fc": [fc], "fd": [fd], "pi": [pi],
                                            "u1cc": [a["u1cc"]], "u2cc": [a['u2cc']], "u1dd": [a['u1dd']], "u2dd": [a['u2dd']], "u1cd": [a['u1cd']], "u2cd": [a['u2cd']], "u1dc": [a['u1dc']], "u2dc": [a['u2dc']],
                                            "CC": [c["CC"]], "DD": [c["DD"]], "CD": [c["CD"]], "DC": [c["DC"]], "P1": [b["p1"]], "P2": [b["p1"]], "Mixed": [c["Mixed"]]})
                            df = pd.concat([df, tempdf])


df.to_csv("point025incV1.csv")

In [ ]:
df = pd.read_csv("point025incV1.csv")
df = df.round(decimals=3)

In [ ]:
mixeddf = df[df['Mixed']]
mixeddf.sort_values('P1')
mixeddf.to_csv("mixed025V1.csv")

In [ ]:
mixeddf = pd.read_csv("mixed025V1.csv")

In [ ]:
mixpimax = mixeddf['pi'].max()
mixpimin = mixeddf['pi'].min()
mixpiunq = mixeddf['pi'].unique()

print(f"pi max = {mixpimax}")
print(f"pi min = {mixpimin}")
print(f"vals include {mixpiunq}")

In [ ]:
ccdf = df[df['CC']==True]
ccpimax = ccdf['pi'].max()
ccpimin = ccdf['pi'].min()
ccpiunq = ccdf['pi'].unique()

print(f"pi max = {ccpimax}")
print(f"pi min = {ccpimin}")
print(f"vals include {ccpiunq}")
ccdf

In [ ]:
cddf = df[df['CD']==True]
cdpimax = cddf['pi'].max()
cdpimin = cddf['pi'].min()
cdpiunq = cddf['pi'].unique()

print(f"pi max = {cdpimax}")
print(f"pi min = {cdpimin}")
print(f"vals include {cdpiunq}")
cddf

In [ ]:
dddf = df[df['DD']]
ddpimax = dddf['pi'].max()
ddpimin = dddf['pi'].min()
ddpiunq = dddf['pi'].unique()

print(f"pi max = {ddpimax}")
print(f"pi min = {ddpimin}")
print(f"vals include {ddpiunq}")
dddf

In [ ]:
dcdf = df[df['DC']]
dcpimax = cddf['pi'].max()
dcpimin = cddf['pi'].min()
dcpiunq = cddf['pi'].unique()

print(f"pi max = {dcpimax}")
print(f"pi min = {dcpimin}")
print(f"vals include {dcpiunq}")
dcdf

In [ ]:
from plotnine import ggplot, aes, geom_point, geom_jitter

df123 = df[(df['pi']==0.6)&(df['theta_upper']==0.9)&(df['theta_lower']==0.1)]

df123CC = df123[df123['CC']==True]
df123DD = df123[df123['DD']==True]
df123CD = df123[df123['CD']==True]
df123DC = df123[df123['DC']==True]

(
    ggplot()  # What data to use
    + aes(x="b1", y="b2")  # What variable to use
    + geom_jitter(df123CC, color="blue", width = 0.02, height = 0.02)  # Geometric object to use for drawing
    + geom_jitter(df123DD, color="red", width = 0.02, height = 0.02)
    + geom_jitter(df123CD, color="green", width = 0.02, height = 0.02)
    + geom_jitter(df123DC, color="black", width = 0.02, height = 0.02)
)